In [65]:
import geopandas as gpd
import pandas as pd
from pystac_client import Client  
from pystac_client import ItemSearch
from pystac.item import Item
import datetime
from shapely.geometry import shape
from rasterio.crs import CRS
import numpy as np
from pathlib import Path
import requests


In [66]:
SEARCH_WINDOW_DAYS = 3

In [67]:
df_rtc = gpd.read_parquet('rtc_s1_data.parquet')
df_rtc.head()

,opera_id,start_time,url_vh,url_vv,track_number,geometry,acq_date
201,OPERA_L2_RTC-S1_T107-227877-IW2_20150318T00354...,2015-03-18T00:35:43Z,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,107,"POLYGON ((-99.32096 33.55928, -98.28645 33.713...",2015-03-18
200,OPERA_L2_RTC-S1_T107-227877-IW2_20150821T00355...,2015-08-21T00:35:50Z,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,107,"POLYGON ((-99.32085 33.55932, -98.33524 33.706...",2015-08-21
199,OPERA_L2_RTC-S1_T107-227877-IW2_20160710T00355...,2016-07-10T00:35:55Z,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,107,"POLYGON ((-99.32052 33.56034, -98.34096 33.706...",2016-07-10
198,OPERA_L2_RTC-S1_T107-227877-IW2_20161014T00355...,2016-10-14T00:35:58Z,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,107,"POLYGON ((-99.32032 33.56018, -98.33776 33.706...",2016-10-14
197,OPERA_L2_RTC-S1_T107-227877-IW2_20161107T00355...,2016-11-07T00:35:58Z,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,https://datapool.asf.alaska.edu/RTC/OPERA-S1/O...,107,"POLYGON ((-99.32140 33.55962, -98.33879 33.705...",2016-11-07


In [68]:
records = df_rtc.to_dict('records')

# Example

In [69]:
r = records[1_000]

sample_geo = r['geometry']
acq_dt = pd.Timestamp(r['acq_date']).to_pydatetime()

In [70]:
stac = 'https://cmr.earthdata.nasa.gov/cloudstac/'    # CMR-STAC API Endpoint
api = Client.open(f'{stac}/LPCLOUD/')
collections = ['OPERA_L3_DIST-ALERT-HLS_V1']

search_params = {"collections": collections,
                 "intersects": sample_geo.__geo_interface__,
                 "datetime": [acq_dt - datetime.timedelta(days=SEARCH_WINDOW_DAYS), 
                              acq_dt + datetime.timedelta(days=SEARCH_WINDOW_DAYS)],
                 "max_items": 100}
search_dist = api.search(**search_params)
l = list(search_dist.items())
len(l)

7

In [71]:
def format_item(item):
    assets_ind = [('dist-veg-ind', value.href) for (key, value) in sorted(item.assets.items()) if ('VEG-IND' in value.href and '.tif' in value.href)]
    out = {**{'id': item.id}, 
           **item.properties, 
           **{'geometry': shape(item.geometry)},
           **{key: value for (key, value) in assets_ind}
          }
    return out
    

In [72]:
r['start_time']

'2023-11-12T15:34:03Z'

In [75]:
l[0]

<Item id=OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1>

In [73]:
data = list(map(format_item, l))
df_resp = pd.DataFrame(data)
df_resp = gpd.GeoDataFrame(df_resp, geometry=df_resp.geometry, crs=CRS.from_epsg(4326))
df_resp['acq_dt'] = pd.to_datetime(df_resp['datetime'])
df_resp['acq_date']= df_resp['acq_dt'].dt.date
df_resp['sensor'] = df_resp['id'].map(lambda oid: oid.split('_')[-3])
df_resp['diff_days'] = ((df_resp['acq_dt'] - pd.Timestamp(r['start_time']))) #/ 60**3 / 24
df_resp.head()

,id,eo:cloud_cover,datetime,start_datetime,end_datetime,geometry,dist-veg-ind,acq_dt,acq_date,sensor,diff_days
0,OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059...,3,2023-11-11T08:30:15.782Z,2023-11-11T08:30:15.782Z,2023-11-11T08:30:15.782Z,"POLYGON ((35.70837 35.11017, 36.91211 35.13690...",https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2023-11-11 08:30:15.782000+00:00,2023-11-11,S2B,-2 days +16:56:12.782000
1,OPERA_L3_DIST-ALERT-HLS_T36SYE_20231111T082059...,6,2023-11-11T08:30:17.311Z,2023-11-11T08:30:17.311Z,2023-11-11T08:30:17.311Z,"POLYGON ((36.39833 35.10722, 36.44042 36.09547...",https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2023-11-11 08:30:17.311000+00:00,2023-11-11,S2B,-2 days +16:56:14.311000
2,OPERA_L3_DIST-ALERT-HLS_T36SYE_20231113T081005...,45,2023-11-13T08:10:05.036Z,2023-11-13T08:10:05.036Z,2023-11-13T08:10:28.931Z,"POLYGON ((36.39833 35.10722, 36.44042 36.09547...",https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2023-11-13 08:10:05.036000+00:00,2023-11-13,L9,0 days 16:36:02.036000
3,OPERA_L3_DIST-ALERT-HLS_T37SBV_20231113T081005...,50,2023-11-13T08:10:05.036Z,2023-11-13T08:10:05.036Z,2023-11-13T08:10:28.931Z,"POLYGON ((35.70837 35.11017, 36.91211 35.13690...",https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2023-11-13 08:10:05.036000+00:00,2023-11-13,L9,0 days 16:36:02.036000
4,OPERA_L3_DIST-ALERT-HLS_T37SCV_20231113T081005...,52,2023-11-13T08:10:05.036Z,2023-11-13T08:10:05.036Z,2023-11-13T08:10:28.931Z,"POLYGON ((36.80483 35.13500, 37.41078 35.14447...",https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2023-11-13 08:10:05.036000+00:00,2023-11-13,L9,0 days 16:36:02.036000


In [74]:
data[0]

{'id': 'OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1',
 'eo:cloud_cover': 3,
 'datetime': '2023-11-11T08:30:15.782Z',
 'start_datetime': '2023-11-11T08:30:15.782Z',
 'end_datetime': '2023-11-11T08:30:15.782Z',
 'geometry': <POLYGON ((35.708 35.11, 36.912 35.137, 36.886 36.126, 35.668 36.099, 35.708...>,
 'dist-veg-ind': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1_VEG-IND.tif'}

In [63]:
def localize_one_file(
    url: str,
    dst_dir: str | Path
) -> Path:
    local_fn = url.split("/")[-1]
    out_path = dst_dir / local_fn
    if not dst_dir.exists():
        dst_dir.mkdir(parents=True)
    if out_path.exists():
        return out_path
        
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(out_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=16384): 
                f.write(chunk)
    return out_path

In [76]:
localize_one_file('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1_VEG-IND.tif', Path('.'))

HTTPError: 403 Client Error: Forbidden for url: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T37SBV_20231111T082059Z_20231223T034114Z_S2B_30_v1_VEG-IND.tif

# Get All Data